In [1]:
import niimpy
import niimpy.preprocessing.location as location
import pandas
import folium

In [45]:
data = niimpy.reading.google_takeout.location_history(
    "takeout-20241111T082655Z-001.zip",
    start_date = pandas.to_datetime("2022-10-05", utc=True),
    end_date = pandas.to_datetime("2022-10-15", utc=True),
)
data = location.filter_location(
    data,
    remove_disabled=False, remove_network=False, remove_zeros=True
)
data.columns

Index(['accuracy', 'source', 'device', 'altitude', 'verticalaccuracy',
       'platformtype', 'servertimestamp', 'devicetimestamp', 'batterycharging',
       'formfactor', 'velocity', 'heading', 'placeid', 'latitude', 'longitude',
       'inferred_latitude', 'inferred_longitude', 'activity_type',
       'activity_inference_confidence', 'user'],
      dtype='object')

In [48]:
location.location_significant_place_features(data)

,n_static,normalized_entropy,n_top5,n_top2,n_top1,n_transitions,n_moving,user,n_top4,n_sps,device,max_dist_home,n_rare,n_home,n_top3,entropy
timestamp,,,,,,,,,,,,,,,,
2022-10-31 00:00:00+00:00,2252.0,3.514595,0.0,73.0,2107.0,12.0,495.0,efee31e4-2bf2-11f0-8ada-efe59ca75eb8,31.0,4.0,1832214273,4105.519218,0.0,1847.0,41.0,4.872263


In [50]:
location.find_home(data["latitude"], data["longitude"], data.index)

(np.float64(60.22987432475728), np.float64(24.775551517475726))

In [33]:
data[["activity_type", "activity_inference_confidence"]].dropna().iloc[20:40]

,activity_type,activity_inference_confidence
timestamp,,
2022-10-09 06:28:06+00:00,UNKNOWN,40.0
2022-10-09 06:28:22+00:00,TILTING,100.0
2022-10-09 06:29:38+00:00,IN_VEHICLE,93.0
2022-10-09 06:30:42+00:00,TILTING,100.0
2022-10-09 06:31:33+00:00,UNKNOWN,40.0
2022-10-09 06:32:36+00:00,TILTING,100.0
2022-10-09 06:32:51+00:00,STILL,62.0
2022-10-09 06:39:52.662000+00:00,TILTING,100.0
2022-10-09 06:40:54+00:00,STILL,97.0


In [ ]:
locations = data[["latitude", "longitude"]].drop_duplicates()
map_center = [locations['latitude'].mean(), locations['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

folium.PolyLine(locations[['latitude', 'longitude']].values, color="blue", weight=2.5, opacity=1).add_to(mymap)

for _, row in locations.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(mymap)

mymap